In [1]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings

#from langchain_community.vectorstores import FAISS

from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


import requests
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
url = 'http://apis.data.go.kr/6260000/FoodService/getFoodKr'


params = {
    'serviceKey': "your api",  # API key parameter
    'numOfRows': 500,        # Number of rows to retrieve
    'pageNo': 1,            # Page number to retrieve
    'resultType': 'json',   # Set result format to JSON (if supported by API)
}

response = requests.get(url, params=params, verify=False)

if response.status_code == 200:
    data = response.json()
    print('API Call is successful!')
else:
    print(response.text)

API Call is successful!


In [6]:
df = pd.DataFrame(data['getFoodKr']['item'])
df.head()

,UC_SEQ,MAIN_TITLE,GUGUN_NM,LAT,LNG,PLACE,TITLE,SUBTITLE,ADDR1,ADDR2,CNTCT_TEL,HOMEPAGE_URL,USAGE_DAY_WEEK_AND_TIME,RPRSNTV_MENU,MAIN_IMG_NORMAL,MAIN_IMG_THUMB,ITEMCNTNTS
0,70,만드리곤드레밥,강서구,35.177387,128.95245,만드리곤드레밥,만드리곤드레밥,,강서구 공항앞길 85번길 13,,051-941-3669,,11:00-21:00\n(20:00 라스트오더),"돌솥곤드레정식, 단호박오리훈제",https://www.visitbusan.net/uploadImgs/files/cn...,https://www.visitbusan.net/uploadImgs/files/cn...,"곤드레밥에는 일반적으로 건조 곤드레나물이 사용되는데,\n이곳은 생 곤드레나물을 사용..."
1,77,민물가든,강서구,35.160550,128.89468,민물가든,민물가든,민물가든,강서구 둔치중앙길5(봉림동),,051-971-8428,https://blog.naver.com/rladba1,24.03.12 ~ 24. 12.31 휴업중\n11:00a.m. ~ 21:00p.m...,"묵은지붕어조림, 붕어찜",https://www.visitbusan.net/uploadImgs/files/cn...,https://www.visitbusan.net/uploadImgs/files/cn...,"30년간 운영해온 생선찜전문점으로, 전통방식인 나무통을 사용하여 조리하는 것이 특징..."
2,94,가야할매밀면,연제구,35.185196,129.07988,가야할매밀면,가야할매밀면,가야할매밀면,부산 연제구 월드컵대로 145번길 32\n,,051-865-8017,,11:00-20:00,"물 밀면, 비빔밀면\n",https://www.visitbusan.net/uploadImgs/files/cn...,https://www.visitbusan.net/uploadImgs/files/cn...,"유명한 밀면전문점으로, 밀가루에 옥수수 전분을 섞어, 다른 밀면전문점들보다 더 탄력..."
3,95,국제밀면본점,연제구,35.196890,129.07785,국제밀면본점,국제밀면본점,,연제구 중앙대로1235번길 23-6,,051-501-5507,,10:00-20:00\n4~9월 10:00-21:00 (라스트오더 20:30 전후)\n,"물밀면, 비빔밀면",https://www.visitbusan.net/uploadImgs/files/cn...,https://www.visitbusan.net/uploadImgs/files/cn...,밀면전문점 중에서도 특히 맛으로 소문난 이곳은\n소 사골만을 사용한 육수 등 독창적...
4,102,할매가야밀면,중구,35.098934,129.03122,할매가야밀면,할매가야밀면,,중구 광복로 56-14,,051-246-3314,,10:30-21:30,"밀면, 비빔밀면",https://www.visitbusan.net/uploadImgs/files/cn...,https://www.visitbusan.net/uploadImgs/files/cn...,"깔끔한 맛의 육수가 인상적인 40년 전통의 밀면 전문점으로, \n식사시간에는 항상 ..."


In [7]:
df.to_csv('./datas/busan_restaurants.csv', index=False, encoding='utf-8-sig')

In [2]:
OPEN_API_KEY = os.environ["OPENAI_API_KEY"]

# Load the CSV file as documents
loader = CSVLoader(file_path='./datas/busan_restaurants.csv', encoding='utf-8-sig')
docs = loader.load()

# Initialize OpenAI embeddings (or another embedding model)
embeddings = OpenAIEmbeddings(openai_api_key=OPEN_API_KEY)

/tmp/ipykernel_32287/2093561314.py:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=OPEN_API_KEY)


In [3]:
# Use FAISS as the vector store for similarity search
vectorstore = FAISS.from_documents(docs, embeddings)

# Initialize the OpenAI LLM (or use another LLM like GPT-4)
llm = OpenAI(openai_api_key=OPEN_API_KEY)

# Set up the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # You can use 'map_reduce' or others if needed
    retriever=vectorstore.as_retriever(),
)

# Query the system with a question related to the CSV data
query = "만드리곤드레밥에서는 무슨 음식을 파나요?"
response = qa_chain.run(query)

# Output the response
print(response)

/tmp/ipykernel_32287/4180398150.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=OPEN_API_KEY)
/tmp/ipykernel_32287/4180398150.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


 곤드레밥


In [4]:
query = "비빔밀면 맛집은 어디인가요?"
response = qa_chain.run(query)

# Output the response
print(response)

 
가장 맛있는 비빔밀면 맛집은 대연밀면이며, 대연밀면은 남구 천제등로 55에 위치하고 있습니다. 
